In [1]:
import os
# from pyspark.sql import SparkSession

# import configparser
# config = configparser.ConfigParser()
# config.read_file(open('cluster.config'))

# spark = SparkSession.builder.getOrCreate()
from pyspark.sql import SparkSession
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = "/opt/conda/bin:/opt/spark-2.4.3-bin-hadoop2.7/bin:/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/lib/jvm/java-8-openjdk-amd64/bin"
os.environ["SPARK_HOME"] = "/opt/spark-2.4.3-bin-hadoop2.7"
os.environ["HADOOP_HOME"] = "/opt/spark-2.4.3-bin-hadoop2.7"

# print(config['AWS']['KEY'])

# os.environ['AWS_ACCESS_KEY_ID']=config['AWS']['AWS_ACCESS_KEY_ID']
# os.environ['AWS_SECRET_ACCESS_KEY']=config['AWS']['AWS_SECRET_ACCESS_KEY']

# AWS_ACCESS_KEY_ID = config['AWS']['KEY']
# AWS_SECRET_ACCESS_KEY = config['AWS']['SECRET']

spark = SparkSession.builder\
.config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
.config("spark.jars.repositories", "https://repos.spark-packages.org/")\
.enableHiveSupport().getOrCreate()



# spark = SparkSession.builder\
# .config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
# .config("spark.jars.repositories", "https://repos.spark-packages.org/")\
# .config("spark.hadoop.fs.s3a.impl","org.apache.hadoop.fs.s3a.S3AFileSystem")\
# .enableHiveSupport().getOrCreate()

# sc = spark.sparkContext
# sc._jsc.hadoopConfiguration().set("fs.s3.access.key", AWS_ACCESS_KEY_ID)
# sc._jsc.hadoopConfiguration().set("fs.s3.secret.key", AWS_SECRET_ACCESS_KEY)

df_spark =spark.read.load('./sas_data')
print("Completed, ran successfully")

Completed, ran successfully


In [2]:
# spark = SparkSession.builder.config("spark.hadoop.fs.s3a.impl","org.apache.hadoop.fs.s3a.S3AFileSystem").getOrCreate()


In [3]:
df_spark.createOrReplaceTempView("imm_table_1")

In [4]:
immigration_df = spark.sql('SELECT cicid AS cicid, \
                          i94yr AS year, \
                          i94mon AS month, \
                          i94cit AS country_code1, \
                          i94res AS country_code2, \
                          i94port AS port_of_entry, \
                          date_add("1960-01-01", arrdate) AS arrival_date, \
                          i94mode AS mode_of_arrival, \
                          i94addr AS state_or_terr, \
                          date_add("1960-01-01", depdate) AS departure_date, \
                          i94bir AS age, \
                          i94visa AS VISA, \
                          occup AS occupation, \
                          entdepa AS arrival_flag, \
                          entdepd AS departure_flag, \
                          entdepu AS update_flag, \
                          matflag AS match_flag, \
                          biryear AS birth_year, \
                          to_date(dtaddto, "MMddyyy") AS date_added, \
                          gender AS gender, \
                          insnum AS ins_number, \
                          airline AS incoming_airline, \
                          admnum AS admission_number, \
                          fltno AS flight_number, \
                          visatype AS visa_type \
                          FROM imm_table_1')

In [5]:
from pyspark.sql.types import IntegerType, DoubleType

In [6]:
imm_df = immigration_df.withColumn('country_code1', immigration_df.country_code1.cast(IntegerType()))\
                       .withColumn('country_code2', immigration_df.country_code2.cast(IntegerType()))\
                       .withColumn('cicid', immigration_df.cicid.cast(IntegerType()))\
                       .withColumn('year', immigration_df.year.cast(IntegerType()))\
                       .withColumn('month', immigration_df.month.cast(IntegerType()))\
                       .withColumn('mode_of_arrival', immigration_df.mode_of_arrival.cast(IntegerType()))\
                       .withColumn('age', immigration_df.age.cast(IntegerType()))\
                       .withColumn('VISA', immigration_df.VISA.cast(IntegerType()))\
                       .withColumn('birth_year', immigration_df.birth_year.cast(IntegerType()))

In [7]:
imm_df.createOrReplaceTempView('imm_clean_col')

In [8]:
null_df = spark.sql("SELECT cast(((COUNT(*) - COUNT(state_or_terr)) / COUNT(*)) * 100 AS decimal(5,2)) AS st, \
                     cast(((COUNT(*) - COUNT(update_flag)) / COUNT(*)) * 100 AS decimal(5,2)) AS uf, \
                     cast(((COUNT(*) - COUNT(cicid)) / COUNT(*)) * 100 AS decimal(5,2)) AS ci, \
                     cast(((COUNT(*) - COUNT(year)) / COUNT(*)) * 100 AS decimal(5,2)) AS year_pc, \
                     cast(((COUNT(*) - COUNT(month)) / COUNT(*)) * 100 AS decimal(5,2)) AS month_pcn, \
                     cast(((COUNT(*) - COUNT(country_code1)) / COUNT(*)) * 100 AS decimal(5,2)) AS cc1_p, \
                     cast(((COUNT(*) - COUNT(country_code2)) / COUNT(*)) * 100 AS decimal(5,2)) AS cc2_p, \
                     cast(((COUNT(*) - COUNT(arrival_date)) / COUNT(*)) * 100 AS decimal(5,2)) AS arrdate, \
                     cast(((COUNT(*) - COUNT(mode_of_arrival)) / COUNT(*)) * 100 AS decimal(5,2)) AS modearr_, \
                     cast(((COUNT(*) - COUNT(departure_date)) / COUNT(*)) * 100 AS decimal(5,2)) AS deptdate, \
                     cast(((COUNT(*) - COUNT(age)) / COUNT(*)) * 100 AS decimal(5,2)) AS age_p, \
                     cast(((COUNT(*) - COUNT(VISA)) / COUNT(*)) * 100 AS decimal(5,2)) AS VISA_pc, \
                     cast(((COUNT(*) - COUNT(occupation)) / COUNT(*)) * 100 AS decimal(5,2)) AS occ_pcnt_null, \
                     cast(((COUNT(*) - COUNT(arrival_flag)) / COUNT(*)) * 100 AS decimal(5,2)) AS arrflag_pcnt_null, \
                     cast(((COUNT(*) - COUNT(departure_flag)) / COUNT(*)) * 100 AS decimal(5,2)) AS deptflag_pcnt_null, \
                     cast(((COUNT(*) - COUNT(match_flag)) / COUNT(*)) * 100 AS decimal(5,2)) AS matchflg_pcnt_null, \
                     cast(((COUNT(*) - COUNT(birth_year)) / COUNT(*)) * 100 AS decimal(5,2)) AS brth_pcnt_null, \
                     cast(((COUNT(*) - COUNT(date_added)) / COUNT(*)) * 100 AS decimal(5,2)) AS dateadd_pcnt_null, \
                     cast(((COUNT(*) - COUNT(gender)) / COUNT(*)) * 100 AS decimal(5,2)) AS gender_pcnt_null, \
                     cast(((COUNT(*) - COUNT(ins_number)) / COUNT(*)) * 100 AS decimal(5,2)) AS ins_pcnt_null, \
                     cast(((COUNT(*) - COUNT(incoming_airline)) / COUNT(*)) * 100 AS decimal(5,2)) AS al_pcnt_null, \
                     cast(((COUNT(*) - COUNT(admission_number)) / COUNT(*)) * 100 AS decimal(5,2)) AS admnum_pcnt_null, \
                     cast(((COUNT(*) - COUNT(flight_number)) / COUNT(*)) * 100 AS decimal(5,2)) AS fn_pcnt_null, \
                     cast(((COUNT(*) - COUNT(visa_type)) / COUNT(*)) * 100 AS decimal(5,2)) AS visatype_pcnt_null \
                     FROM imm_clean_col")

null_df.show()

+----+-----+----+-------+---------+-----+-----+-------+--------+--------+-----+-------+-------------+-----------------+------------------+------------------+--------------+-----------------+----------------+-------------+------------+----------------+------------+------------------+
|  st|   uf|  ci|year_pc|month_pcn|cc1_p|cc2_p|arrdate|modearr_|deptdate|age_p|VISA_pc|occ_pcnt_null|arrflag_pcnt_null|deptflag_pcnt_null|matchflg_pcnt_null|brth_pcnt_null|dateadd_pcnt_null|gender_pcnt_null|ins_pcnt_null|al_pcnt_null|admnum_pcnt_null|fn_pcnt_null|visatype_pcnt_null|
+----+-----+----+-------+---------+-----+-----+-------+--------+--------+-----+-------+-------------+-----------------+------------------+------------------+--------------+-----------------+----------------+-------------+------------+----------------+------------+------------------+
|4.93|99.99|0.00|   0.00|     0.00| 0.00| 0.00|   0.00|    0.01|    4.60| 0.03|   0.00|        99.74|             0.01|              4.47|          

In [9]:
imm_df.show()

+-------+----+-----+-------------+-------------+-------------+------------+---------------+-------------+--------------+---+----+----------+------------+--------------+-----------+----------+----------+----------+------+----------+----------------+----------------+-------------+---------+
|  cicid|year|month|country_code1|country_code2|port_of_entry|arrival_date|mode_of_arrival|state_or_terr|departure_date|age|VISA|occupation|arrival_flag|departure_flag|update_flag|match_flag|birth_year|date_added|gender|ins_number|incoming_airline|admission_number|flight_number|visa_type|
+-------+----+-----+-------------+-------------+-------------+------------+---------------+-------------+--------------+---+----+----------+------------+--------------+-----------+----------+----------+----------+------+----------+----------------+----------------+-------------+---------+
|5748517|2016|    4|          245|          438|          LOS|  2016-04-30|              1|           CA|    2016-05-08| 40|   1| 

In [10]:
imm_df_final = imm_df.drop('occupation', 'update_flag', 'ins_number')

In [11]:
imm_df_final.show()

+-------+----+-----+-------------+-------------+-------------+------------+---------------+-------------+--------------+---+----+------------+--------------+----------+----------+----------+------+----------------+----------------+-------------+---------+
|  cicid|year|month|country_code1|country_code2|port_of_entry|arrival_date|mode_of_arrival|state_or_terr|departure_date|age|VISA|arrival_flag|departure_flag|match_flag|birth_year|date_added|gender|incoming_airline|admission_number|flight_number|visa_type|
+-------+----+-----+-------------+-------------+-------------+------------+---------------+-------------+--------------+---+----+------------+--------------+----------+----------+----------+------+----------------+----------------+-------------+---------+
|5748517|2016|    4|          245|          438|          LOS|  2016-04-30|              1|           CA|    2016-05-08| 40|   1|           G|             O|         M|      1976|2016-10-29|     F|              QF|  9.495387003E10| 

In [12]:
imm_df_final.count()

3096313

In [13]:
imm_df_final.printSchema()

root
 |-- cicid: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- country_code1: integer (nullable = true)
 |-- country_code2: integer (nullable = true)
 |-- port_of_entry: string (nullable = true)
 |-- arrival_date: date (nullable = true)
 |-- mode_of_arrival: integer (nullable = true)
 |-- state_or_terr: string (nullable = true)
 |-- departure_date: date (nullable = true)
 |-- age: integer (nullable = true)
 |-- VISA: integer (nullable = true)
 |-- arrival_flag: string (nullable = true)
 |-- departure_flag: string (nullable = true)
 |-- match_flag: string (nullable = true)
 |-- birth_year: integer (nullable = true)
 |-- date_added: date (nullable = true)
 |-- gender: string (nullable = true)
 |-- incoming_airline: string (nullable = true)
 |-- admission_number: double (nullable = true)
 |-- flight_number: string (nullable = true)
 |-- visa_type: string (nullable = true)



In [14]:
file = open('./I94_SAS_Labels_Descriptions.SAS', 'r')
content = file.readlines()

In [15]:
lines = []

for i in range(9, 298):
    lines.append(content[i])

my_data = []
for i in range(len(lines)):
    my_data.append(lines[i].split('='))
    
for i in range(len(my_data)):
    my_data[i] = [my_data[i][0].strip(), my_data[i][1].strip().strip('\'')]
    
columns = ['country_code', 'country']

from pyspark.sql.types import StructType, StructField, StringType, IntegerType

schema = StructType([\
                    StructField('country_code', StringType()),
                    StructField('country', StringType())])

country_codes_df = spark.createDataFrame(data = my_data, schema = schema)

country_codes_df.show(truncate = False)


+------------+---------------------------------------------------------+
|country_code|country                                                  |
+------------+---------------------------------------------------------+
|582         |MEXICO Air Sea, and Not Reported (I-94, no land arrivals)|
|236         |AFGHANISTAN                                              |
|101         |ALBANIA                                                  |
|316         |ALGERIA                                                  |
|102         |ANDORRA                                                  |
|324         |ANGOLA                                                   |
|529         |ANGUILLA                                                 |
|518         |ANTIGUA-BARBUDA                                          |
|687         |ARGENTINA                                                |
|151         |ARMENIA                                                  |
|532         |ARUBA                                

In [16]:
country_codes_df.printSchema()

root
 |-- country_code: string (nullable = true)
 |-- country: string (nullable = true)



In [17]:
country_df_final = country_codes_df.withColumn('country_code', country_codes_df.country_code.cast(IntegerType()))

In [18]:
country_df_final.printSchema()

root
 |-- country_code: integer (nullable = true)
 |-- country: string (nullable = true)



In [19]:
# Even though the country table only has a country code and a cooresponding name, it is important 
# to use a separate table for them in case later on, other country information can be added to the table
# in the form of additional columns.

In [20]:
port_lines = []
for i in range(302, 963):
    port_lines.append(content[i])

In [21]:
print(port_lines[0])

   'ALC'	=	'ALCAN, AK             '



In [22]:
my_data2 = []

# for line in port_lines:
#     my_data2.append(line.split('='))

for i in range(0, 516):
    my_data2.append(port_lines[i].split('='))
    
city_state = []
row = 303
index = 0



my_data2[28][1] = my_data2[28][1].strip().strip('\'').strip()
my_data2[28][1] = my_data2[28][1].replace(' ', ', ')


# print(my_data2[516][1])
my_data2[76][1] = 'WASHINGTON DC, DC'
my_data2[49][1] = 'PACIFIC COAST HWY STATION, CA'
my_data2[217][1] = 'WARROAD INTL SPB, MN'
my_data2[417][1] = 'HULL FIELD SUGAR LAND ARPT, TX'
my_data2[385][1] = 'BLACK HILLS SPEARFISH, SD'
my_data2[387][1] = 'SAIPAN, NORTHERN MARIANA ISLANDS'
my_data2[428][1] = 'PASO DEL NORTE, TX'
my_data2[444][1] = 'CRUZ BAY ST JOHN, VI'
my_data2[478][1] = 'BELLINGHAM, WA #INTL'
my_data2.append(port_lines[577].split('='))

for i in range(579, 591):
    my_data2.append(port_lines[i].split('='))

# for port in my_data2:
#     print('row: ' + str(row) + ' index: ' +str(index) + ' ' + port[1].split(', ')[1])
#     row = row + 1
#     index = index + 1
    

for i in range(len(my_data2)):
    my_data2[i] = [my_data2[i][0].strip().strip('\''), my_data2[i][1].strip().strip('\'').strip().split(', ')[0], my_data2[i][1].strip().strip('\'').strip().split(', ')[1]]
    
for data_point in my_data2:
    if len(data_point[2]) > 2:
        print(data_point[2])
    
# print(my_data2)



AR (BPS)
CA (BPS)
CA (BPS)
CA (BPS)
CO #ARPT
FL #ARPT
LA (BPS)
ME (BPS)
MT (BPS)
NM (BPS)
NM (BPS)
NM (BPS)
SC #ARPT
NORTHERN MARIANA ISLANDS
TX (BPS)
TX (BPS)
VA #ARPT
VT (I-91)
VT (RT. 5)
VT (BP - SECTOR HQ)
WA #INTL
WA (BPS)


In [23]:
for row in my_data2:
    if (len(row[2]) == 2 or row[2] == 'NORTHERN MARIANA ISLANDS'):
        row.append(None)
    else:
#         row.append(row[2].split(' ')[1])
        last_col = row[2].split(' ', 1)
        row[2] = last_col[0]
        row.append(last_col[1])
        print(last_col)
        
        
# print(my_data2)


['AR', '(BPS)']
['CA', '(BPS)']
['CA', '(BPS)']
['CA', '(BPS)']
['CO', '#ARPT']
['FL', '#ARPT']
['LA', '(BPS)']
['ME', '(BPS)']
['MT', '(BPS)']
['NM', '(BPS)']
['NM', '(BPS)']
['NM', '(BPS)']
['SC', '#ARPT']
['TX', '(BPS)']
['TX', '(BPS)']
['VA', '#ARPT']
['VT', '(I-91)']
['VT', '(RT. 5)']
['VT', '(BP - SECTOR HQ)']
['WA', '#INTL']
['WA', '(BPS)']


In [24]:
for item in my_data2:
    if len(item) != 4:
        print(item)

In [25]:
us_port_schema = StructType([\
                    StructField('port_code', StringType()),
                    StructField('port_name', StringType()),
                    StructField('state_code', StringType()),
                    StructField('extra_info', StringType())])

us_port_df = spark.createDataFrame(data = my_data2, schema = us_port_schema)

In [26]:
us_port_df.show(300)
us_port_df.printSchema()

+---------+--------------------+----------+----------+
|port_code|           port_name|state_code|extra_info|
+---------+--------------------+----------+----------+
|      ALC|               ALCAN|        AK|      null|
|      ANC|           ANCHORAGE|        AK|      null|
|      BAR|BAKER AAF - BAKER...|        AK|      null|
|      DAC|       DALTONS CACHE|        AK|      null|
|      PIZ|DEW STATION PT LA...|        AK|      null|
|      DTH|        DUTCH HARBOR|        AK|      null|
|      EGL|               EAGLE|        AK|      null|
|      FRB|           FAIRBANKS|        AK|      null|
|      HOM|               HOMER|        AK|      null|
|      HYD|               HYDER|        AK|      null|
|      JUN|              JUNEAU|        AK|      null|
|      5KE|           KETCHIKAN|        AK|      null|
|      KET|           KETCHIKAN|        AK|      null|
|      MOS|MOSES POINT INTER...|        AK|      null|
|      NIK|             NIKISKI|        AK|      null|
|      NOM

In [27]:
state_lines = []
for i in range(981, 1036):
    state_lines.append(content[i])

In [28]:
print(state_lines[0])

	'AL'='ALABAMA'



In [29]:
my_data3 = []
for line in state_lines:
    split_data = line.split('=')
    my_data3.append([split_data[0].strip().strip('\''), split_data[1].strip().strip('\'')])
        
    
    
print(my_data3)
    


[['AL', 'ALABAMA'], ['AK', 'ALASKA'], ['AZ', 'ARIZONA'], ['AR', 'ARKANSAS'], ['CA', 'CALIFORNIA'], ['CO', 'COLORADO'], ['CT', 'CONNECTICUT'], ['DE', 'DELAWARE'], ['DC', 'DIST. OF COLUMBIA'], ['FL', 'FLORIDA'], ['GA', 'GEORGIA'], ['GU', 'GUAM'], ['HI', 'HAWAII'], ['ID', 'IDAHO'], ['IL', 'ILLINOIS'], ['IN', 'INDIANA'], ['IA', 'IOWA'], ['KS', 'KANSAS'], ['KY', 'KENTUCKY'], ['LA', 'LOUISIANA'], ['ME', 'MAINE'], ['MD', 'MARYLAND'], ['MA', 'MASSACHUSETTS'], ['MI', 'MICHIGAN'], ['MN', 'MINNESOTA'], ['MS', 'MISSISSIPPI'], ['MO', 'MISSOURI'], ['MT', 'MONTANA'], ['NC', 'N. CAROLINA'], ['ND', 'N. DAKOTA'], ['NE', 'NEBRASKA'], ['NV', 'NEVADA'], ['NH', 'NEW HAMPSHIRE'], ['NJ', 'NEW JERSEY'], ['NM', 'NEW MEXICO'], ['NY', 'NEW YORK'], ['OH', 'OHIO'], ['OK', 'OKLAHOMA'], ['OR', 'OREGON'], ['PA', 'PENNSYLVANIA'], ['PR', 'PUERTO RICO'], ['RI', 'RHODE ISLAND'], ['SC', 'S. CAROLINA'], ['SD', 'S. DAKOTA'], ['TN', 'TENNESSEE'], ['TX', 'TEXAS'], ['UT', 'UTAH'], ['VT', 'VERMONT'], ['VI', 'VIRGIN ISLANDS'], ['

In [30]:
state_schema = StructType([\
                          StructField('state_code', StringType()),
                          StructField('state_name', StringType())])

In [31]:
state_df_final = spark.createDataFrame(data = my_data3, schema = state_schema)

In [32]:
state_df_final.show()

+----------+-----------------+
|state_code|       state_name|
+----------+-----------------+
|        AL|          ALABAMA|
|        AK|           ALASKA|
|        AZ|          ARIZONA|
|        AR|         ARKANSAS|
|        CA|       CALIFORNIA|
|        CO|         COLORADO|
|        CT|      CONNECTICUT|
|        DE|         DELAWARE|
|        DC|DIST. OF COLUMBIA|
|        FL|          FLORIDA|
|        GA|          GEORGIA|
|        GU|             GUAM|
|        HI|           HAWAII|
|        ID|            IDAHO|
|        IL|         ILLINOIS|
|        IN|          INDIANA|
|        IA|             IOWA|
|        KS|           KANSAS|
|        KY|         KENTUCKY|
|        LA|        LOUISIANA|
+----------+-----------------+
only showing top 20 rows



In [33]:
airport_df = spark.read.options(delimiter = ',').option("header", True).csv('airport-codes_csv.csv')

In [34]:
airport_df.show()

+-----+-------------+--------------------+------------+---------+-----------+----------+------------+--------+---------+----------+--------------------+
|ident|         type|                name|elevation_ft|continent|iso_country|iso_region|municipality|gps_code|iata_code|local_code|         coordinates|
+-----+-------------+--------------------+------------+---------+-----------+----------+------------+--------+---------+----------+--------------------+
|  00A|     heliport|   Total Rf Heliport|          11|       NA|         US|     US-PA|    Bensalem|     00A|     null|       00A|-74.9336013793945...|
| 00AA|small_airport|Aero B Ranch Airport|        3435|       NA|         US|     US-KS|       Leoti|    00AA|     null|      00AA|-101.473911, 38.7...|
| 00AK|small_airport|        Lowell Field|         450|       NA|         US|     US-AK|Anchor Point|    00AK|     null|      00AK|-151.695999146, 5...|
| 00AL|small_airport|        Epps Airpark|         820|       NA|         US|     

In [35]:
airport_df = airport_df.filter(airport_df.iso_country == 'US')
airport_df.count()

22757

In [36]:
from pyspark.sql.functions import split

airport_df_final = airport_df.withColumn('state_code', split(airport_df.iso_region, '-')[1])\
                             .withColumn('elevation_ft', airport_df.elevation_ft.cast(IntegerType()))\
                             .drop('iso_region')\
                             .drop('continent')\
                             .drop('iso_country')

airport_df_final.show()
airport_df_final.printSchema()

+-----+-------------+--------------------+------------+------------+--------+---------+----------+--------------------+----------+
|ident|         type|                name|elevation_ft|municipality|gps_code|iata_code|local_code|         coordinates|state_code|
+-----+-------------+--------------------+------------+------------+--------+---------+----------+--------------------+----------+
|  00A|     heliport|   Total Rf Heliport|          11|    Bensalem|     00A|     null|       00A|-74.9336013793945...|        PA|
| 00AA|small_airport|Aero B Ranch Airport|        3435|       Leoti|    00AA|     null|      00AA|-101.473911, 38.7...|        KS|
| 00AK|small_airport|        Lowell Field|         450|Anchor Point|    00AK|     null|      00AK|-151.695999146, 5...|        AK|
| 00AL|small_airport|        Epps Airpark|         820|     Harvest|    00AL|     null|      00AL|-86.7703018188476...|        AL|
| 00AR|       closed|Newport Hospital ...|         237|     Newport|    null|     n

In [37]:
from pyspark.sql.functions import upper, col

In [38]:
us_cities_demog_df = spark.read.options(delimiter = ';').option("header", True).csv('us-cities-demographics.csv')

city_dem_df = us_cities_demog_df.withColumn('Median_Age', us_cities_demog_df['Median Age'].cast(DoubleType()))\
                                .withColumn('City', upper(col('City')))\
                                .withColumn('State', upper(col('State')))\
                                .withColumn('Male_Population', us_cities_demog_df['Male Population'].cast(IntegerType()))\
                                .withColumn('Female_Population', us_cities_demog_df['Female Population'].cast(IntegerType()))\
                                .withColumn('Total_Population', us_cities_demog_df['Total Population'].cast(IntegerType()))\
                                .withColumn('Number_of_Veterans', us_cities_demog_df['Number of Veterans'].cast(IntegerType()))\
                                .withColumn('Foreign-born', us_cities_demog_df['Foreign-born'].cast(IntegerType()))\
                                .withColumn('Average_Household_Size', us_cities_demog_df['Average Household Size'].cast(DoubleType()))\
                                .withColumn('Count', us_cities_demog_df['Count'].cast(IntegerType()))\
                                .drop('Median Age')\
                                .drop('Male Population')\
                                .drop('Female Population')\
                                .drop('Total Population')\
                                .drop('Number of Veterans')\
                                .drop('Average Household Size')\
                                .drop('State Code')

city_dem_df.show()
city_dem_df.printSchema()

+----------------+--------------+------------+--------------------+------+----------+---------------+-----------------+----------------+------------------+----------------------+
|            City|         State|Foreign-born|                Race| Count|Median_Age|Male_Population|Female_Population|Total_Population|Number_of_Veterans|Average_Household_Size|
+----------------+--------------+------------+--------------------+------+----------+---------------+-----------------+----------------+------------------+----------------------+
|   SILVER SPRING|      MARYLAND|       30908|  Hispanic or Latino| 25924|      33.8|          40601|            41862|           82463|              1562|                   2.6|
|          QUINCY| MASSACHUSETTS|       32935|               White| 58723|      41.0|          44129|            49500|           93629|              4147|                  2.39|
|          HOOVER|       ALABAMA|        8229|               Asian|  4759|      38.5|          38040|    

In [39]:
# state_df_final.write.mode("overwrite").parquet(path="./state_table.parquet")

# state_df_final.write.mode("overwrite").parquet(path='s3://capstonebucket1743/test1/')

In [40]:
# city_dem_df.write.mode("overwrite").parquet(path="./city_dem_table.parquet")

In [41]:
# us_port_df.write.mode("overwrite").parquet(path="./us_port_table.parquet")

In [42]:
# airport_df_final.write.mode("overwrite").parquet(path="./airport_table.parquet")

In [43]:
# country_df_final.write.mode("overwrite").parquet(path="./country_table.parquet")

In [44]:
# imm_df_final.write.mode("overwrite").parquet(path="./immigration_table.parquet")

In [45]:
state_df_final.createOrReplaceTempView('state_stage')
city_dem_df.createOrReplaceTempView('city_stage')
us_port_df.createOrReplaceTempView('port_stage')
airport_df_final.createOrReplaceTempView('airport_stage')
country_df_final.createOrReplaceTempView('country_stage')
imm_df_final.createOrReplaceTempView('imm_stage')

In [46]:
state_df_final.show(5)
city_dem_df.show(5)
us_port_df.show(5)
airport_df_final.show(5)
country_df_final.show(5)
imm_df_final.show(5)

+----------+----------+
|state_code|state_name|
+----------+----------+
|        AL|   ALABAMA|
|        AK|    ALASKA|
|        AZ|   ARIZONA|
|        AR|  ARKANSAS|
|        CA|CALIFORNIA|
+----------+----------+
only showing top 5 rows

+----------------+-------------+------------+--------------------+-----+----------+---------------+-----------------+----------------+------------------+----------------------+
|            City|        State|Foreign-born|                Race|Count|Median_Age|Male_Population|Female_Population|Total_Population|Number_of_Veterans|Average_Household_Size|
+----------------+-------------+------------+--------------------+-----+----------+---------------+-----------------+----------------+------------------+----------------------+
|   SILVER SPRING|     MARYLAND|       30908|  Hispanic or Latino|25924|      33.8|          40601|            41862|           82463|              1562|                   2.6|
|          QUINCY|MASSACHUSETTS|       32935|      

In [47]:
data_model2 = spark.sql("SELECT SUM(`Foreign-born`) AS total_foreign_born, City \
                         FROM city_stage \
                         GROUP BY City \
                         ORDER BY total_foreign_born DESC")

In [54]:
data_model2.show(20)

+------------------+-------------+
|total_foreign_born|         City|
+------------------+-------------+
|          16062500|     NEW YORK|
|           7427125|  LOS ANGELES|
|           3481050|      HOUSTON|
|           2867315|      CHICAGO|
|           2007465|     SAN JOSE|
|           1869210|    SAN DIEGO|
|           1634125|       DALLAS|
|           1503510|      PHOENIX|
|           1485995|SAN FRANCISCO|
|           1303945|        MIAMI|
|           1040230|  SAN ANTONIO|
|           1026695| PHILADELPHIA|
|            950615|       BOSTON|
|            908430|       AUSTIN|
|            798545|      EL PASO|
|            778215|     GLENDALE|
|            764995|    SANTA ANA|
|            717020|   FORT WORTH|
|            685665|      ANAHEIM|
|            680592|      HIALEAH|
+------------------+-------------+
only showing top 20 rows



In [55]:
data_model = spark.sql("SELECT COUNT(*) AS num_im_thru_port, p.port_name  \
                        FROM imm_stage i JOIN port_stage p ON i.port_of_entry = p.port_code \
                        GROUP BY p.port_name \
                        ORDER BY num_im_thru_port DESC") 

In [56]:
data_model.show(20)

+----------------+----------------+
|num_im_thru_port|       port_name|
+----------------+----------------+
|          485916|        NEW YORK|
|          343941|           MIAMI|
|          310163|     LOS ANGELES|
|          152586|   SAN FRANCISCO|
|          149195|         ORLANDO|
|          142720|        HONOLULU|
|          136122|NEWARK/TETERBORO|
|          130564|         CHICAGO|
|          101481|         HOUSTON|
|           95977| FORT LAUDERDALE|
|           92579|         ATLANTA|
|           89280|       LAS VEGAS|
|           80919|           AGANA|
|           74835|   WASHINGTON DC|
|           71809|          DALLAS|
|           57354|          BOSTON|
|           47719|         SEATTLE|
|           38890|         PHOENIX|
|           37832|         DETROIT|
|           25632|           TAMPA|
+----------------+----------------+
only showing top 20 rows



In [62]:
data_model3 = spark.sql("WITH table1 AS (SELECT SUM(`Foreign-born`) AS total_foreign_born, City FROM city_stage GROUP BY City), \
                               table2 AS (SELECT COUNT(*) AS num_im_thru_port, p.port_name \
                               FROM imm_stage i JOIN port_stage p ON i.port_of_entry = p.port_code \
                               GROUP BY p.port_name ORDER BY num_im_thru_port DESC)\
                               SELECT table1.total_foreign_born, table1.City, table2.num_im_thru_port, CAST(((table2.num_im_thru_port/(3096313)) * 100) AS DECIMAL(5, 2)) AS percent_im \
                               FROM table1 JOIN table2 ON table1.City = table2.port_name \
                               ORDER BY 1 DESC")

In [63]:
data_model3.show()

+------------------+-------------+----------------+----------+
|total_foreign_born|         City|num_im_thru_port|percent_im|
+------------------+-------------+----------------+----------+
|          16062500|     NEW YORK|          485916|     15.69|
|           7427125|  LOS ANGELES|          310163|     10.02|
|           3481050|      HOUSTON|          101481|      3.28|
|           2867315|      CHICAGO|          130564|      4.22|
|           2007465|     SAN JOSE|            4490|      0.15|
|           1869210|    SAN DIEGO|           10944|      0.35|
|           1634125|       DALLAS|           71809|      2.32|
|           1503510|      PHOENIX|           38890|      1.26|
|           1485995|SAN FRANCISCO|          152586|      4.93|
|           1303945|        MIAMI|          343941|     11.11|
|           1040230|  SAN ANTONIO|            7066|      0.23|
|           1026695| PHILADELPHIA|           24973|      0.81|
|            950615|       BOSTON|           57354|    

In [53]:
# spark.stop()